# Visualize cell volumes of optimized structures
**Instruction**  
The aim of this notebook is to visualize the cell volumes of optimized structures.  
The following code requires the result csv file of `2_calc-elastic-tensors.ipynb` executed on Matlantis.  

In [ ]:
# Import Library
import pandas as pd
import matplotlib.pyplot as plt
from utility import *
import openpyxl
import numpy as np

In [ ]:
# Read calclated data
df_pfp_angew = prep_df('FILE_PATH.csv')
df_chgnet_angew = prep_df('FILE_PATH.csv')
# df_chgnet_x23 = prep_df()
print(df_pfp_angew.shape)
print(df_chgnet_angew.shape)
df_pfp_angew.head(2)

In [ ]:
from ase import io
def get_density(atoms):
    cell_volume = atoms.get_volume()
    total_mass = sum(atoms.get_masses())
    density = total_mass / cell_volume * 1.66053906660
    return density

def density_columns(df_calc, mode):
    opt_dens_list, exp_dens_list = [], []
    for refcode in df_calc['refcode'].tolist():
        atoms = io.read(f'dataset_spackman/{refcode}.cif')
        dens_exp = get_density(atoms)
        if mode=='PFP':
            atoms = io.read(f'opt_cifs_PFP/{refcode}_opt.cif')
        elif mode=='chgnet':
            atoms = io.read(f'opt_cifs_chgnet/{refcode}_opt.cif')
        dens_opt = get_density(atoms)
        opt_dens_list.append(dens_opt)
        exp_dens_list.append(dens_exp)
    df_calc['opt_density'] = opt_dens_list
    df_calc['exp_density'] = exp_dens_list
    
density_columns(df_pfp_angew, 'PFP')
density_columns(df_chgnet_angew, 'chgnet')

In [ ]:
df_pfp_angew[['refcode', 'exp_V', 'opt_V', 'deltaV']].head()

In [ ]:
# Visualize lattice change (set filename!!)
vis_opt_lattice(df=df_pfp_angew, color='skyblue', save=True, save_name='SAVE_FIG_NAME.png')
vis_opt_lattice(df=df_chgnet_angew, color='gray', save=True, save_name='SAVE_FIG_NAME.png')

In [ ]:
# Read HF results
data = openpyxl.load_workbook('../dataset/ange202110716-sup-0001-si/S-HF-3c-elastic-tensors.xlsx')
name = data.sheetnames[1]
sheet = data[name]
deltaV_list, refcode_list = [], []
for i in np.arange(4,48):
    deltaV = sheet.cell(row=i, column=6).value
    refcode = sheet.cell(row=i, column=3).value
    deltaV_list.append(deltaV)
    refcode_list.append(refcode)
    
df_hf_angew = pd.DataFrame({'refcode': refcode_list, 'deltaV': deltaV_list})
df_hf_angew.head(2)

In [ ]:
df_pfp_angew = arrange_hf(df_pfp_angew, df_hf_angew)
df_chgnet_angew = arrange_hf(df_chgnet_angew, df_hf_angew)

In [ ]:
plt.rcParams["font.family"] = 'Arial'
plt.rcParams["font.size"] = 13

size = 80
alpha = 1
ec = 'k'

plt.figure(figsize=(6,5))
plt.scatter(df_pfp_angew['exp_V'], df_pfp_angew['opt_V_HF'], 
            label='S-HF-3c', ec=ec, c='orange', s=size, alpha=alpha)
plt.scatter(df_chgnet_angew['exp_V'], df_chgnet_angew['opt_V'], 
            label='CHGNet', c='gray', ec=ec, s=size, alpha=alpha)
plt.scatter(df_pfp_angew['exp_V'], df_pfp_angew['opt_V'], 
            label='PFP', c='skyblue', ec=ec, s=size, alpha=alpha)
plt.plot([0, 8000], [0, 8000], c='k', linestyle='dashed')
plt.legend()
plt.xlabel(r'exp. $V$ ($\mathrm{\mathring{A}}^3$)')
plt.ylabel(r'opt. $V$ ($\mathrm{\mathring{A}}^3$)')
plt.tight_layout()

plt.savefig('SAVE_FIG_NAME.png', dpi=300)

In [ ]:
plt.rcParams["font.family"] = 'Arial'
plt.rcParams["font.size"] = 15
size = 100
plt.figure(figsize=(3.5,3))
plt.scatter(df_pfp_angew['exp_V'], df_pfp_angew['opt_V_HF'], 
            label='S-HF-3c', ec=ec, c='orange', s=size, alpha=alpha)
plt.scatter(df_chgnet_angew['exp_V'], df_chgnet_angew['opt_V'], 
            label='CHGNet', c='gray', ec=ec, s=size, alpha=alpha)
plt.scatter(df_pfp_angew['exp_V'], df_pfp_angew['opt_V'], 
            label='PFP', c='skyblue', ec=ec, s=size, alpha=alpha)
plt.plot([0, 8000], [0, 8000], c='k', linestyle='dashed')
# plt.legend()
plt.xlabel(r'exp. $V$ ($\mathrm{\mathring{A}}^3$)')
plt.ylabel(r'opt. $V$ ($\mathrm{\mathring{A}}^3$)')
plt.xlim(0, 2000)
plt.ylim(0, 2000)
plt.tight_layout()
plt.savefig('SAVE_FIG_NAME.png', dpi=300)